<a href="https://colab.research.google.com/github/WayneGretzky1/nba_game_predictor/blob/main/nba_game_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import numpy as np
import torch

In [64]:
nba_df = pd.read_csv("https://raw.githubusercontent.com/WayneGretzky1/nba_game_predictor/refs/heads/main/cleaned.csv")
nba_df

,gameid,gamedate,home_teamid,away_teamid,home_win,away_win,home_win_avg_prev5,away_win_avg_prev5,home_points_avg_prev5,away_points_avg_prev5,...,home_freethrowsattempted_avg_prev5,away_freethrowsattempted_avg_prev5,home_freethrowspercentage_avg_prev5,away_freethrowspercentage_avg_prev5,home_reboundstotal_avg_prev5,away_reboundstotal_avg_prev5,home_foulspersonal_avg_prev5,away_foulspersonal_avg_prev5,home_turnovers_avg_prev5,away_turnovers_avg_prev5
0,22100127,2021-11-05,1610612761,1610612739,False,True,0.4,0.6,102.8,99.4,...,21.2,22.8,0.7074,0.7750,46.2,47.6,20.0,16.0,14.4,17.4
1,22100137,2021-11-06,1610612756,1610612737,True,False,0.6,0.2,108.2,102.0,...,21.8,16.0,0.7802,0.7342,48.2,50.6,17.8,17.2,15.0,13.2
2,22100145,2021-11-07,1610612744,1610612745,True,False,0.8,0.0,111.8,102.6,...,14.8,23.8,0.6820,0.6300,51.8,46.4,18.2,19.4,17.6,21.0
3,22100150,2021-11-08,1610612742,1610612740,True,False,0.8,0.2,108.4,96.0,...,20.6,18.6,0.6490,0.8328,44.2,49.2,20.0,18.4,12.6,20.8
4,22100154,2021-11-08,1610612747,1610612766,True,False,0.8,0.2,110.4,105.4,...,16.4,20.8,0.7542,0.7388,42.6,39.6,18.8,17.0,16.8,13.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4669,22401195,2025-04-13,1610612750,1610612762,True,False,0.8,0.0,122.6,110.0,...,25.8,22.2,0.8206,0.8264,45.2,41.2,18.6,18.0,11.8,11.4
4670,22401191,2025-04-13,1610612755,1610612741,False,True,0.0,0.6,107.2,127.2,...,16.4,22.0,0.7026,0.8088,40.4,42.8,17.0,17.4,13.8,14.2
4671,22401200,2025-04-13,1610612758,1610612756,True,False,0.2,0.0,108.8,105.8,...,16.8,17.0,0.8130,0.7944,42.0,38.6,19.2,19.8,12.8,12.2
4672,22401187,2025-04-13,1610612738,1610612766,True,False,0.8,0.0,116.8,97.2,...,15.6,16.8,0.7824,0.7564,50.8,47.8,12.8,14.2,11.2,17.0


In [71]:
#choose features X and label y
X = nba_df.drop(columns=['home_win', 'away_win', 'gameid', 'gamedate', 'home_teamid', 'away_teamid']).to_numpy() #every col except these
y = nba_df['home_win'].to_numpy()

In [72]:
#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
#normalize with standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
# print(X_train_norm)

In [74]:
#PyTorch Dataset class
class NumpyDataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, input_array, target_output_array):
    'Initialization'
    self.input_tensor = torch.from_numpy(input_array).float()
    self.output_tensor = torch.from_numpy(target_output_array).float()

  def __len__(self):
      'Denotes the total number of samples'
      return len(self.input_tensor)

  def __getitem__(self, index):
      'Generates one sample of data'
      # Load data and get label
      X = self.input_tensor[index]
      y = self.output_tensor[index]

      return X, y

In [75]:
#Convert to PyTorch Dataset
training_dataset = NumpyDataset(X_train_norm, y_train)
testing_dataset = NumpyDataset(X_test_norm, y_test)
